In [1]:
#on_kaggle = True
on_kaggle = False

#TRAIN_PREDICT = 'predict'
TRAIN_PREDICT = 'train'

if not on_kaggle:
    import os
    os.environ["CUDA_VISIBLE_DEVICES"]="3"

SEED = 42

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import gc
import os
from scipy.stats import spearmanr
from math import floor, ceil
import random

from unidecode import unidecode
import re

import tensorflow as tf
import tensorflow.keras.backend as K
print(tf.__version__)

#fix bug with using CuDNNLSTM
#gpus = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(gpus[0], True)

from transformers import *
import transformers
print(transformers.__version__)

np.set_printoptions(suppress=True)

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    #torch.manual_seed(seed)
    #torch.cuda.manual_seed_all(seed)
    #torch.backends.cudnn.deterministic = True
    tf.random.set_seed(seed)
seed_everything(SEED)

2.0.0
2.3.0


#### 1. Read data and tokenizer

Read tokenizer and data, as well as defining the maximum sequence length that will be used for the input to Bert (maximum is usually 512 tokens)

In [3]:
if on_kaggle:
    PATH = '../input/google-quest-challenge/'
    BERT_PATH = '../input/bert-base-uncased-huggingface-transformer/'
    CKPT_LOAD_PATH = ['../input/google-quest-qa-labeling-checkpoints-v3/fold0-epoch3.h5py', 
                      '../input/google-quest-qa-labeling-checkpoints-v3/fold1-epoch3.h5py',
                      '../input/google-quest-qa-labeling-checkpoints-v3/fold2-epoch3.h5py',
                      '../input/google-quest-qa-labeling-checkpoints-v3/fold3-epoch3.h5py',
                      '../input/google-quest-qa-labeling-checkpoints-v3/fold4-epoch3.h5py'
                     ]
    tokenizer = BertTokenizer.from_pretrained(BERT_PATH+'bert-base-uncased-vocab.txt')
else:##offline
    PATH = '../data/'
    BERT_PATH = '../model/xlnet-base-cased-v3'
    CKPT_SAVE_PATH = '../checkpoint/xlnet-base-cased-v3/'
    CKPT_LOAD_PATH = ['../checkpoint/xlnet-base-cased/']
    tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

MAX_SEQUENCE_LENGTH = 512

df_train = pd.read_csv(PATH+'train.csv')
df_test = pd.read_csv(PATH+'test.csv')
df_sub = pd.read_csv(PATH+'sample_submission.csv')
print('train shape =', df_train.shape)
print('test shape =', df_test.shape)

output_categories = list(df_train.columns[11:])
input_categories = list(df_train.columns[[1,2,5,9,10]])#9:category, 10:host
print('\noutput categories:\n\t', output_categories)
print('\ninput categories:\n\t', input_categories)

train shape = (6079, 41)
test shape = (476, 11)

output categories:
	 ['question_asker_intent_understanding', 'question_body_critical', 'question_conversational', 'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer', 'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice', 'question_type_compare', 'question_type_consequence', 'question_type_definition', 'question_type_entity', 'question_type_instructions', 'question_type_procedure', 'question_type_reason_explanation', 'question_type_spelling', 'question_well_written', 'answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance', 'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 'answer_type_reason_explanation', 'answer_well_written']

input categories:
	 ['question_title', 'question_body', 'answer', 'category', 'host']


#### 2. Preprocessing functions

These are some functions that will be used to preprocess the raw text data into useable Bert inputs.

In [12]:
# t,q,a = df_train[input_categories].iloc[3].to_list()
# ids = tokenizer.encode_plus(a, None, 'longest_first', max_length=None)

# input_ids = ids['input_ids']
# len(input_ids)

#input_segments = ids['token_type_ids']
# for i in range(5):
#     t,q,a = df_train.loc[i, ['question_title', 'question_body', 'answer']]
#     print(t)
#     print('-'*10)
#     print(q)
#     print('-'*10)
#     print(a)
#     print('='*40)

# t = df_train.loc[df_train.host=='stackoverflow.com', 'question_title'].values[11]
# q = df_train.loc[df_train.host=='stackoverflow.com', 'question_body'].values[11]

# tokens = tokenizer.encode_plus(t+' '+q, None, add_special_tokens=True, max_length=512, truncation_strategy='longest_first')
# tokens['token_type_ids']
# print(t+'END OF TITLE\n'+q)
# print(' '.join(tokenizer.ids_to_tokens[i] for i in tokens['input_ids']))
# [tokenizer.ids_to_tokens[i] for i in tokens['input_ids']]

In [4]:
def count_special_char(x):
    special_character = re.compile(r'[^A-Za-z\.\-\?\!\,\#\@\% ]', re.IGNORECASE)
    x_ascii = unidecode(x)
    s = special_character.findall(x_ascii)
    #print(s)
    c = len(s)
    return c

def count_cap_char(x):
    c = sum(1 for l in x if l.isupper())
    return c

def count_unique_words(x):
    """returns a ratio"""
    special_character = re.compile(r'[^A-Za-z0-9]', re.IGNORECASE)
    a = [w for w in special_character.split(x) if w!='']
    r = len(set(a))/len(a)
    return r

#print(t)
#count_cap_char(t), len(t)
#count_special_char(t), len(t)
#string.printable

#from textblob import TextBlob
# print(t.lower())
# print('='*40)
# print(TextBlob(t.lower()).correct())

In [5]:
l = df_train.category.unique().tolist()
category2index = dict([(l[i],i+1) for i in range(len(l))])
category2index['UNK'] = 0

l = df_train.host.unique().tolist()
host2index = dict([(l[i],i+1) for i in range(len(l))])
host2index['UNK'] = 0

category2index

{'LIFE_ARTS': 1,
 'CULTURE': 2,
 'SCIENCE': 3,
 'STACKOVERFLOW': 4,
 'TECHNOLOGY': 5,
 'UNK': 0}

In [17]:
class BucketedDataIterator(tf.keras.utils.Sequence):
    """
    tr = BucketedDataIterator(df_train_trans, num_buckets=10, testset=False)
    """
    def __init__(self, df, num_buckets=10, testset=False):
        df = df.sort_values('qa_len').reset_index(drop=True)
        self.df = df
        self.size = len(df) / num_buckets
        self.dfs = []
        for bucket in range(num_buckets):
            self.dfs.append(df.loc[bucket*self.size: (bucket+1)*self.size - 1])
        self.num_buckets = num_buckets

        # cursor[i] will be the cursor for the ith bucket
        self.cursor = np.array([0] * num_buckets)
        self.shuffle()

        self.epochs = 0
        
        self.testset = testset

    def shuffle(self):
        #sorts dataframe by sequence length, but keeps it random within the same length
        for i in range(self.num_buckets):
            self.dfs[i] = self.dfs[i].sample(frac=1).reset_index(drop=True)
            self.cursor[i] = 0
    
    def on_epoch_end(self):
        self.cursor = np.array([0] * num_buckets)
        self.shuffle()
    
    def __len__(self):
        return ceil(len(self.df) / BATCH_SIZE)

    def __getitem__(self, idx):
        to_choose = []
        for i in range(self.num_buckets):
            cursor = self.cursor[i]+BATCH_SIZE+1
            if cursor < self.size:
                to_choose.append(i)
        if to_choose!=[]:
            i = np.random.choice(to_choose, 1)[0]
            #print(i)
        else:
            i = np.random.choice(list(range(self.num_buckets)), 1)[0]
            self.shuffle()
            #print('Empty list Error\ni = np.random.choice(to_choose, 1)[0]\n')
            #print(i)

        res = self.dfs[i].loc[self.cursor[i]:self.cursor[i]+BATCH_SIZE-1]
        self.cursor[i] += BATCH_SIZE

        # Pad sequences with 0s so they are all the same length
        #qa_tokens, qa_masks, qa_segments
        maxlen = max(res['qa_len'])
        tokens = np.zeros([BATCH_SIZE, maxlen], dtype=np.int32)
        for i, x_i in enumerate(tokens):
            x_i[:res['qa_len'].values[i]] = res['qa_tokens'].values[i]
        masks = np.zeros([BATCH_SIZE, maxlen], dtype=np.float32)
        for i, x_i in enumerate(masks):
            x_i[:res['qa_len'].values[i]] = res['qa_masks'].values[i]
        segments = np.zeros([BATCH_SIZE, maxlen], dtype=np.int32)
        for i, x_i in enumerate(segments):
            x_i[:res['qa_len'].values[i]] = res['qa_segments'].values[i]
        if maxlen>1024:
            tokens = tokens[:1024]
            masks = masks[:1024]
            segments = segments[:1024]
        
#         if idx==0:
#             print(np.array([tokens, masks, segments]))
#             print('='*40)
#             print(res[output_categories].values)
        
        if self.testset:
            return [tokens, masks, segments]
        return [tokens, masks, segments], [res[output_categories].values]

In [168]:
tr = BucketedDataIterator(df_train_trans)
x,y = tr[0]
x[0].shape, y[0].shape, len(tr), tr.cursor

((8, 288), (8, 30), 760, array([0, 0, 0, 8, 0, 0, 0, 0, 0, 0]))

In [7]:
# def _convert_to_transformer_inputs(title, question, answer, tokenizer):
#     """Converts tokenized input to ids, masks and segments for transformer (including bert)
    
#     NOTE: USE Head + Tail truncation
#     """
    
#     def return_id(str1, None):

#         inputs = tokenizer.encode_plus(str1, None,
#             add_special_tokens=True,
#             max_length=None,
#             truncation_strategy='longest_first')
        
#         length = MAX_SEQUENCE_LENGTH
        
#         input_ids =  inputs["input_ids"]
#         input_masks = [1.0] * len(input_ids)
#         input_segments = inputs["token_type_ids"]
#         if len(input_ids)>length:#Head + Tail truncate
#             input_ids = input_ids[:128] + input_ids[-384:]
#             input_masks = input_masks[:128] + input_masks[-384:]
#             input_segments = input_segments[:128] + input_segments[-384:]
#         padding_length = length - len(input_ids)
#         padding_id = tokenizer.pad_token_id
#         input_ids = input_ids + ([padding_id] * padding_length)
#         input_masks = input_masks + ([0.0] * padding_length)
#         input_segments = input_segments + ([0] * padding_length)
        
#         return [input_ids, input_masks, input_segments]
    
#     input_ids_q, input_masks_q, input_segments_q = return_id(title + ' ' + question, None)
    
#     input_ids_a, input_masks_a, input_segments_a = return_id(answer, None)

#     return [input_ids_q, input_masks_q, input_segments_q,
#             input_ids_a, input_masks_a, input_segments_a]

# def compute_input_arrays(df, columns, tokenizer):
#     input_ids_q, input_masks_q, input_segments_q = [], [], []
#     input_ids_a, input_masks_a, input_segments_a = [], [], []
#     special_char_q, special_char_a = [], []
#     cap_char_q, cap_char_a = [], []
#     unique_words_q, unique_words_a = [], []
#     category_index, host_index = [], []
#     for _, instance in tqdm(df[columns].iterrows()):
#         t, q, a = instance.question_title, instance.question_body, instance.answer
        
#         ids_q, masks_q, segments_q, ids_a, masks_a, segments_a = _convert_to_transformer_inputs(t, q, a, tokenizer)
        
#         input_ids_q.append(ids_q)
#         input_masks_q.append(masks_q)
#         input_segments_q.append(segments_q)

#         input_ids_a.append(ids_a)
#         input_masks_a.append(masks_a)
#         input_segments_a.append(segments_a)
        
#         special_char_q.append(count_special_char(t+q)/len(t+q))
#         special_char_a.append(count_special_char(a)/len(a))
#         cap_char_q.append(count_cap_char(t+q)/len(t+q))
#         cap_char_a.append(count_cap_char(a)/len(a))
#         unique_words_q.append(count_unique_words(t+q))
#         unique_words_a.append(count_unique_words(a))
        
#         category, host = instance.category, instance.host
#         category_index.append([category2index.get(category, 0)])
#         host_index.append([host2index.get(host, 0)])
        
#     return [np.asarray(input_ids_q, dtype=np.int32), 
#             np.asarray(input_masks_q, dtype=np.float32), 
#             np.asarray(input_segments_q, dtype=np.int32),
#             np.asarray(input_ids_a, dtype=np.int32), 
#             np.asarray(input_masks_a, dtype=np.float32), 
#             np.asarray(input_segments_a, dtype=np.int32),
#             np.asarray([special_char_q, special_char_a, cap_char_q, cap_char_a, unique_words_q, unique_words_a], dtype=np.float32).T, 
#             np.asarray(category_index, dtype=np.int32),
#             np.asarray(host_index, dtype=np.int32)
#            ]

# def compute_output_arrays(df, columns):
#     return np.asarray(df[columns])

In [6]:
def make_data_from_df(df, testset=False):
    q_len, a_len = [], []
    q_tokens, a_tokens = [], []
    q_masks, a_masks = [], []
    q_segments, a_segments = [], []
    for _, instance in tqdm(df[['question_title', 'question_body', 'answer']].iterrows()):
        t, q, a = instance.question_title, instance.question_body, instance.answer
        ## Question
        inputs = tokenizer.encode_plus(t+' '+q, None,
                        add_special_tokens=True,
                        max_length=None,
                        truncation_strategy='longest_first')#t+' '+q+' '+a
        tokens = inputs["input_ids"]
        q_tokens.append(tokens)
        q_masks.append([1.0]*len(tokens))
        q_segments.append(inputs["token_type_ids"])
        q_len.append(len(tokens))
        ## Answer
        inputs = tokenizer.encode_plus(a, None,
                        add_special_tokens=True,
                        max_length=None,
                        truncation_strategy='longest_first')#t+' '+q+' '+a
        tokens = inputs["input_ids"]
        a_tokens.append(tokens)
        a_masks.append([1.0]*len(tokens))
        a_segments.append(inputs["token_type_ids"])
        a_len.append(len(tokens))

    ##
    #cols = ['q_tokens', 'q_masks', 'q_segments', 'a_tokens', 'a_masks', 'a_segments', 'q_len', 'a_len'] + \
    #        input_categories + output_categories
    output = pd.DataFrame({'q_tokens':q_tokens, 'q_masks':q_masks, 'q_segments':q_segments, 'q_len':q_len, 
                           'a_tokens':a_tokens, 'a_masks':a_masks, 'a_segments':a_segments, 'a_len':a_len})
    if not testset:
        for col in output_categories:
            output[col] = df[col]
    return output

df_train_trans = make_data_from_df(df_train)
#df_test_trans = make_data_from_df(df_test, testset=True)

In [30]:
#np.quantile(df_train_trans.q_len.values, [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]), \
#np.quantile(df_train_trans.a_len.values, [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95])
#df_train_trans.head(5)

l = []
#df = df_train_trans[['q_len', 'a_len']].sample(frac=1.0)
#df = df_train_trans[['q_len', 'a_len']].sort_values(['q_len'], ascending=True)
df = df_train_trans[['q_len', 'a_len']].copy()
df['qa_len'] = df['q_len'] + df['a_len']
df = df.sort_values(['qa_len'], ascending=True)

df = df.values
for i in range(760):
    v = df[i*8:(i+1)*8, :].max(axis=0)
    l.append(v)

l = np.array(l)

In [37]:
(l[:,0]<512).mean()


0.7776315789473685

#### 3. Create model

`compute_spearmanr()` is used to compute the competition metric for the validation set
<br><br>
`CustomCallback()` is a class which inherits from `tf.keras.callbacks.Callback` and will compute and append validation score and validation/test predictions respectively, after each epoch.
<br><br>
`bert_model()` contains the actual architecture that will be used to finetune BERT to our dataset. It's simple, just taking the sequence_output of the bert_layer and pass it to an AveragePooling layer and finally to an output layer of 30 units (30 classes that we have to predict)
<br><br>
`train_and_predict()` this function will be run to train and obtain predictions

In [15]:
def compute_spearmanr_ignore_nan(trues, preds):
    rhos = []
    for tcol, pcol in zip(np.transpose(trues), np.transpose(preds)):
        rhos.append(spearmanr(tcol, pcol).correlation)
    return np.nanmean(rhos)


class CustomCallback(tf.keras.callbacks.Callback):
    
    def __init__(self, valid_iter, fold=None, stage2=False):

        self.valid_iter = valid_iter
        self.fold = fold
        self.stage2 = stage2
        
    def on_train_begin(self, logs={}):
        self.valid_predictions = []
        
    def on_epoch_end(self, epoch, logs={}):
        self.valid_predictions.append(
            self.model.predict_generator(generator=self.valid_iter)
        )
        
        rho_val = compute_spearmanr_ignore_nan(
            self.valid_outputs, np.average(self.valid_predictions, axis=0))
        
        print("\nvalidation rho: %.4f" % rho_val)
        
        if self.fold is not None and epoch>1:
            if self.stage2:
                self.model.save_weights(CKPT_SAVE_PATH+f'fold{fold}-epoch{epoch}-stage2.h5py')
            else:
                self.model.save_weights(CKPT_SAVE_PATH+f'fold{fold}-epoch{epoch}.h5py')


# def xlnet_model():
#     q_id = tf.keras.layers.Input((None,), dtype=tf.int32)#MAX_SEQUENCE_LENGTH
#     a_id = tf.keras.layers.Input((None,), dtype=tf.int32)
    
#     q_mask = tf.keras.layers.Input((None,), dtype=tf.float32)
#     a_mask = tf.keras.layers.Input((None,), dtype=tf.float32)
    
#     q_atn = tf.keras.layers.Input((None,), dtype=tf.int32)
#     a_atn = tf.keras.layers.Input((None,), dtype=tf.int32)
    
#     feats = tf.keras.layers.Input((6,), dtype=tf.float32)#set dim of additional numeric feats
#     category_index = tf.keras.layers.Input((1,), dtype=tf.int32)
#     host_index = tf.keras.layers.Input((1,), dtype=tf.int32)
    
#     #config = XLNetConfig.from_json_file('../data/xlnet-base-cased/xlnet-base-cased-config.json')
#     #config.output_hidden_states = False # Set to True to obtain hidden states
#     # caution: when using e.g. XLNet, XLNetConfig() will automatically use xlnet-large config
    
#     # normally ".from_pretrained('bert-base-uncased')", but because of no internet, the 
#     # pretrained model has been downloaded manually and uploaded to kaggle.
#     #base_model = bert_base_model()
#     #base_model.load_weights(CKPT_SAVE_PATH+f'fold{fold}-epoch3.h5py')
#     #bert_weights = base_model.layers[8].get_weights()
#     if on_kaggle:
#         roberta_layer = TFBertModel.from_pretrained(BERT_PATH+'bert-base-uncased-tf_model.h5', config=config)
#     else:
#         xlnet_layer = TFXLNetModel.from_pretrained('xlnet-base-cased')#config=config
#     #bert_layer.set_weights(bert_weights)
#     #bert_layer.trainable = False
    
#     # if config.output_hidden_states = True, obtain hidden states via bert_model(...)[-1]
#     q_embedding = xlnet_layer(q_id, attention_mask=q_mask, token_type_ids=q_atn)[0]
#     a_embedding = xlnet_layer(a_id, attention_mask=a_mask, token_type_ids=a_atn)[0]
    
#     #q_embedding = tf.keras.layers.SpatialDropout1D(0.2)(q_embedding)
#     #a_embedding = tf.keras.layers.SpatialDropout1D(0.2)(a_embedding)
    
#     qa_embedding = tf.keras.layers.Concatenate()([q_embedding, a_embedding])
#     qa1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1024, return_sequences=True, 
#                                                             activation='tanh', recurrent_activation='sigmoid', 
#                                                             recurrent_dropout=0, unroll=False, use_bias=False
#                                                             ))(qa_embedding)
#     qa2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(512, return_sequences=True, 
#                                                             activation='tanh', recurrent_activation='sigmoid', 
#                                                             recurrent_dropout=0, unroll=False, use_bias=False
#                                                             ))(qa1)
    
#     qa1_avgpool = tf.keras.layers.GlobalAveragePooling1D()(qa1)
#     qa2_avgpool = tf.keras.layers.GlobalAveragePooling1D()(qa2)
#     #qa_maxpool = tf.keras.layers.GlobalMaxPool1D()(qa)
#     q_avgpool = tf.keras.layers.GlobalAveragePooling1D()(q_embedding)
#     a_avgpool = tf.keras.layers.GlobalAveragePooling1D()(a_embedding)
    
#     category_embed_layer = tf.keras.layers.Embedding(6, 8, input_length=1)
#     host_embed_layer = tf.keras.layers.Embedding(64, 8, input_length=1)
#     cat_embed = category_embed_layer(category_index)
#     host_embed = host_embed_layer(host_index)
#     cat_embed = tf.keras.layers.Flatten()(cat_embed)
#     host_embed = tf.keras.layers.Flatten()(host_embed)
    
#     #x = tf.keras.layers.Concatenate()([q, a, feats, cat_embed, host_embed])
#     x = tf.keras.layers.Concatenate()([qa1_avgpool, qa2_avgpool, q_avgpool, a_avgpool,
#                                        feats, cat_embed, host_embed])
    
#     x = tf.keras.layers.Dropout(0.2)(x)
    
#     x = tf.keras.layers.Dense(30, activation='sigmoid')(x)

#     model = tf.keras.models.Model(inputs=[q_id, q_mask, q_atn, a_id, a_mask, a_atn, 
#                                           feats, category_index, host_index, ], outputs=x)
    
#     return model

def xlnet_model():
    qa_id = tf.keras.layers.Input((None,), dtype=tf.int32)#MAX_SEQUENCE_LENGTH
    qa_mask = tf.keras.layers.Input((None,), dtype=tf.float32)
    qa_atn = tf.keras.layers.Input((None,), dtype=tf.int32)
    
    if on_kaggle:
        roberta_layer = TFBertModel.from_pretrained(BERT_PATH+'bert-base-uncased-tf_model.h5', config=config)
    else:
        xlnet_layer = TFXLNetModel.from_pretrained('xlnet-base-cased')#config=config
    
    # if config.output_hidden_states = True, obtain hidden states via bert_model(...)[-1]
    qa_embedding = xlnet_layer(qa_id, attention_mask=qa_mask, token_type_ids=qa_atn)[0]
    
    #qa_embedding = tf.keras.layers.Concatenate()([q_embedding, a_embedding])
    qa1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1024, return_sequences=True, 
                                                            activation='tanh', recurrent_activation='sigmoid', 
                                                            recurrent_dropout=0, unroll=False, use_bias=False
                                                            ))(qa_embedding)
    qa2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(512, return_sequences=True, 
                                                            activation='tanh', recurrent_activation='sigmoid', 
                                                            recurrent_dropout=0, unroll=False, use_bias=False
                                                            ))(qa1)
    
    qa1_avgpool = tf.keras.layers.GlobalAveragePooling1D()(qa1)
    qa2_avgpool = tf.keras.layers.GlobalAveragePooling1D()(qa2)
    #qa_maxpool = tf.keras.layers.GlobalMaxPool1D()(qa)
    qa_avgpool = tf.keras.layers.GlobalAveragePooling1D()(qa_embedding)
        
    x = tf.keras.layers.Concatenate()([qa1_avgpool, qa2_avgpool, qa_avgpool])
    
    x = tf.keras.layers.Dropout(0.2)(x)
    
    x = tf.keras.layers.Dense(30, activation='sigmoid')(x)

    model = tf.keras.models.Model(inputs=[qa_id, qa_mask, qa_atn, ], outputs=x)
    
    return model


def train(model, train_iter, valid_iter, 
          learning_rate, epochs, batch_size, loss_function, fold, stage2=False):
    """
    multipliers = {'dense_1': 0.5, 'dense_2': 0.4}
    optimizer = LearningRateMultiplier(tf.keras.optimizers.Adam, 
                                        lr_multiplier=multipliers, learning_rate=learning_rate)
    
    """
    custom_callback = CustomCallback(
        valid_iter=valid_iter, 
        fold=fold, stage2=stage2)

    #decay_steps = 1000
    #lr_decayed_fn = tf.keras.experimental.CosineDecay(learning_rate, decay_steps)
    #clr = CyclicLR(base_lr=2e-5, max_lr=5e-5, step_size=2000., mode='triangular')

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)#clipvalue=0.5
    model.compile(loss=loss_function, optimizer=optimizer)
    model.fit_generator(generator=train_iter, epochs=epochs, 
              callbacks=[custom_callback], shuffle=False)#batch_size=batch_size
    
    return custom_callback

def predict(model, test_data, load_weights_path):
    model.load_weights(load_weights_path)
    return model.predict(test_data, batch_size=BATCH_SIZE)

In [89]:
#config = RobertaConfig()
#print(config)
#config.output_hidden_states = False
#xlnet_layer = TFXLNetModel.from_pretrained('xlnet-base-cased')
model = xlnet_model()
model.summary()

#base_model.load_weights('../checkpoint/bert-base-uncased-v3/fold0-epoch3.h5py')

#bert_weights = base_model.layers[8].get_weights()

#new_bert_layer = TFBertModel.from_pretrained('bert-base-uncased', config=config)
#new_bert_layer.set_weights(bert_weights)

#inputs[1][0]

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
tfxl_net_model_1 (TFXLNetModel) ((None, None, 768),) 116718336   input_7[0][0]                    
____________________________________________________________________________________________

#### 4. Obtain inputs and targets, as well as the indices of the train/validation splits

In [59]:
# if TRAIN_PREDICT == 'train':
#     outputs = compute_output_arrays(df_train, output_categories)
#     inputs = compute_input_arrays(df_train, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)

In [11]:
import pickle

#bug: if create for the first time, need to run twice to load it
if TRAIN_PREDICT == 'train':
    if not os.path.isfile('../data/gkf%d.pkl'%SEED):
        print('Create gkf')
        gkf = GroupKFold(n_splits=5).split(X=df_train.question_body, groups=df_train.question_body)
        with open('../data/gkf%d.pkl'%SEED, 'wb') as f:
            pickle.dump(list(gkf), f)
    else:
        print('Load gkf')
        with open('../data/gkf%d.pkl'%SEED, 'rb') as f:
            gkf = pickle.load(f)

Load gkf


In [12]:
#print(len(inputs), inputs[6].shape, inputs[7], inputs[8])
#inputs[6][:,4:]
# for i,(train_idx,val_idx) in enumerate(gkf):
#     if i==0:
#         break
# train_idx[20:30]

#gkf0 = list(gkf)

#### 5. Training, validation and testing

Loops over the folds in gkf and trains each fold for 5 epochs --- with a learning rate of 1e-5 and batch_size of 8. A simple binary crossentropy is used as the objective-/loss-function. 

In [12]:
NUM_EPOCHS = 4
BATCH_SIZE = 8
LearningRate = 3e-5

In [13]:
def custom_bce_loss(y_true, y_pred):
    """
    nunique = [df_train[col].nunique() for col in output_categories]#count unique values of each column
    weights_dict = {5:0.5, 9:1.0, 17:1.5, 3:0.5}
    weights = [weights_dict[i] for i in nunique]
    """
    weights = tf.convert_to_tensor([1. , 1. , 0.5, 0.5, 0.5, 0.5, 1. , 1. , 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1. , 1. , 1. , 1. , 1. , 1.5,
       0.5, 0.5, 0.5, 1. ], dtype=float)
    weights = weights/K.mean(weights)
    loss = tf.multiply(y_true, K.log(y_pred+K.epsilon())) + tf.multiply((1-y_true), K.log(1-y_pred+K.epsilon()))
    loss = tf.multiply(loss, weights)
    bce_loss = tf.reduce_mean(-loss)
    return bce_loss
#     y_true_clip = K.clip(y_true, K.epsilon(), 1)
#     y_pred_clip = K.clip(y_pred, K.epsilon(), 1)
#     kl_loss = tf.reduce_mean(tf.reduce_sum(y_true_clip * K.log(y_true_clip / y_pred_clip), axis=0))
#     return bce_loss*0.9 + kl_loss*0.1

#y_pred = tf.random.uniform((8, 30))
#y_true = tf.random.uniform((8, 30))

#custom_bce_loss(y_true, y_pred)

In [14]:
## training
## if stage2, modify 2 places
if TRAIN_PREDICT == 'train':
    histories = []
    for fold, (train_idx, valid_idx) in enumerate(gkf):

        # will actually only do 1 fold (out of 5) to compare models
        if fold ==0:
            print('========training fold %d========'%fold)
            K.clear_session()
            
            ##stage 1, finetuning bert as embedding
            #model = bert_base_model()
            ##stage 2, load bert weights from stage 1, train full model
            model = xlnet_model()

            #prepare dataset
#             train_inputs = [inputs[i][train_idx] for i in range(len(inputs))]
#             train_outputs = outputs[train_idx]
#             valid_inputs = [inputs[i][valid_idx] for i in range(len(inputs))]
#             valid_outputs = outputs[valid_idx]
            train_iter = BucketedDataIterator(df_train_trans.loc[train_idx, ], num_buckets=10, testset=False)
            valid_iter = BucketedDataIterator(df_train_trans.loc[valid_idx, ], num_buckets=10, testset=False)

            # history contains two lists of valid and test preds respectively:
            #  [valid_predictions_{fold}, test_predictions_{fold}]
            history = train(model, 
                            train_iter=train_iter, 
                            valid_iter=valid_iter, 
                            learning_rate=LearningRate, epochs=NUM_EPOCHS, batch_size=None,
                            loss_function=custom_bce_loss, fold=fold, 
                            stage2=False)#'binary_crossentropy'
            histories.append(history)

# ## training full trainset to lift LB score in the final
# if TRAIN_PREDICT == 'train':
#     histories = []
#     print('========Start training========')
#     K.clear_session()
#     model = bert_model()

#     train_inputs = inputs
#     train_outputs = outputs

#     history = train_and_predict(model, 
#                       train_data=(train_inputs, train_outputs), 
#                       valid_data=None,
#                       test_data=None, 
#                       learning_rate=LearningRate, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE,
#                       loss_function='binary_crossentropy', fold=fold)

#     histories.append(history)

========training fold 0========
Epoch 1/4


ResourceExhaustedError: OOM when allocating tensor with shape[623,623,8,12] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:AddV2] name: model/tfxl_net_model/transformer/layer_._6/rel_attn/add/

In [16]:
#more ideas todo: 
#1.1 modify model
#1.2 OOV words spelling correction
#2. loss, ranking loss?
#3. add custom new tokens?(e.g stackoverflow)
#4. roberta?alxnet?
#5. RankGauss average folds?
#6. freeze some layers of bert?
"""
CV history
---------------
#### bert-base ####
Epoch 5/15
4856/4863 [============================>.] - ETA: 0s - loss: 0.3091
validation rho: 0.3923
4863/4863 [==============================] - 339s 70ms/sample - loss: 0.3091
------LB=0.346

switch to HuggingFace
----------------------
SEEMS DEPENDS ON THE SEED!!!
----------------------------
t + q[:1/2], q[1/2:], a
same

category + host + t + q, category + host + a
same


CUSTOM LOSS
Epoch 4/4
4856/4863 [============================>.] - ETA: 0s - loss: 0.3612
validation rho: 0.3989
4863/4863 [==============================] - 711s 146ms/sample - loss: 0.3612

Epoch 4/4
4856/4863 [============================>.] - ETA: 0s - loss: 0.3597
validation rho: 0.3998
4863/4863 [==============================] - 724s 149ms/sample - loss: 0.3597

add 2 feats
Epoch 4/4
4856/4863 [============================>.] - ETA: 0s - loss: 0.3616
validation rho: 0.4049
4863/4863 [==============================] - 718s 148ms/sample - loss: 0.3616

4 feats
Epoch 4/4
4856/4863 [============================>.] - ETA: 0s - loss: 0.3592
validation rho: 0.4027
4863/4863 [==============================] - 706s 145ms/sample - loss: 0.3591

add cat+host embed dim=16
Epoch 4/4
4856/4863 [============================>.] - ETA: 0s - loss: 0.3594
validation rho: 0.4005
4863/4863 [==============================] - 698s 144ms/sample - loss: 0.3594

embed dim=8
Epoch 4/4
4856/4863 [============================>.] - ETA: 0s - loss: 0.3592
validation rho: 0.4071
4863/4863 [==============================] - 734s 151ms/sample - loss: 0.3592

add LSTM features
validation rho: 0.4081 --fold0
validation rho: 0.4050 --fold1
validation rho: 0.4138 --fold2


--------
epochs progression:

validation rho: 0.3810
validation rho: 0.3974
validation rho: 0.4035
validation rho: 0.4057

"""

'\nCV history\n---------------\n#### bert-base ####\nEpoch 5/15\n4856/4863 [============================>.] - ETA: 0s - loss: 0.3091\nvalidation rho: 0.3923\n4863/4863 [==============================] - 339s 70ms/sample - loss: 0.3091\n------LB=0.346\n\nswitch to HuggingFace\n----------------------\nSEEMS DEPENDS ON THE SEED!!!\n----------------------------\nt + q[:1/2], q[1/2:], a\nsame\n\ncategory + host + t + q, category + host + a\nsame\n\n\nCUSTOM LOSS\nEpoch 4/4\n4856/4863 [============================>.] - ETA: 0s - loss: 0.3612\nvalidation rho: 0.3989\n4863/4863 [==============================] - 711s 146ms/sample - loss: 0.3612\n\nEpoch 4/4\n4856/4863 [============================>.] - ETA: 0s - loss: 0.3597\nvalidation rho: 0.3998\n4863/4863 [==============================] - 724s 149ms/sample - loss: 0.3597\n\nadd 2 feats\nEpoch 4/4\n4856/4863 [============================>.] - ETA: 0s - loss: 0.3616\nvalidation rho: 0.4049\n4863/4863 [==============================] - 718s 

#### 6. Process and submit test predictions

First the test predictions are read from the list of lists of `histories`. Then each test prediction list (in lists) is averaged. Then a mean of the averages is computed to get a single prediction for each data point. Finally, this is saved to `submission.csv`

In [17]:
if TRAIN_PREDICT == 'predict':
    
    test_inputs = compute_input_arrays(df_test, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)
    
    model = bert_model()
    
    test_predictions = [predict(model, test_inputs, load_weights_path=ckpt_load_path) 
                        for ckpt_load_path in CKPT_LOAD_PATH]
    #test_predictions = [np.average(test_predictions[i], axis=0) for i in range(len(test_predictions))]
    test_predictions = np.mean(test_predictions, axis=0)

    df_sub.iloc[:, 1:] = test_predictions

    df_sub.to_csv('submission.csv', index=False)

In [18]:
df_sub.head(10)

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.950409,0.667482,0.239416,0.339715,0.570079,0.466379,0.700281,0.700085,0.571651,...,0.939537,0.927281,0.502933,0.972254,0.975164,0.780713,0.031082,0.050007,0.861596,0.923908
1,46,0.859322,0.471540,0.005667,0.772744,0.760206,0.914244,0.574795,0.459414,0.150252,...,0.614215,0.965300,0.622804,0.984339,0.987231,0.901024,0.941507,0.119569,0.096480,0.902197
2,70,0.908096,0.620519,0.014831,0.787047,0.934155,0.962700,0.608684,0.413993,0.300243,...,0.878956,0.914537,0.562891,0.970561,0.969091,0.786442,0.036871,0.066868,0.924306,0.903338
3,132,0.894989,0.414773,0.009222,0.709674,0.744439,0.904797,0.542779,0.426597,0.098071,...,0.731927,0.948509,0.699444,0.967164,0.983113,0.905921,0.857659,0.183874,0.640012,0.921505
4,200,0.921447,0.422136,0.035422,0.858205,0.736690,0.841480,0.635920,0.601400,0.237748,...,0.662975,0.926064,0.677840,0.978456,0.973675,0.848344,0.341229,0.161605,0.517119,0.905178
5,245,0.949044,0.838918,0.037897,0.670364,0.950395,0.876493,0.649106,0.485590,0.183473,...,0.928902,0.988580,0.675213,0.990091,0.993232,0.954224,0.005941,0.106807,0.949878,0.933668
6,257,0.895700,0.486271,0.006919,0.726316,0.739863,0.893735,0.538539,0.457449,0.074345,...,0.751802,0.954958,0.695134,0.972706,0.987342,0.906162,0.864686,0.182657,0.509210,0.922592
7,267,0.967291,0.712311,0.267039,0.739393,0.795763,0.796656,0.677582,0.649118,0.201059,...,0.906255,0.908691,0.674458,0.960482,0.976234,0.818040,0.005299,0.006220,0.976141,0.915883
8,284,0.892987,0.445296,0.006820,0.724650,0.789642,0.890839,0.530727,0.467378,0.412352,...,0.803386,0.972898,0.646042,0.983505,0.991843,0.919998,0.803075,0.159615,0.551880,0.917416
9,292,0.969639,0.732235,0.017571,0.876691,0.911313,0.860330,0.687019,0.573530,0.061705,...,0.913862,0.876258,0.632740,0.962797,0.977443,0.785966,0.316395,0.109399,0.727858,0.927811


In [19]:
# ##post process
# cols2process = df_train.columns.tolist()[11:]
# print(len(cols2process))

In [20]:
# for col in tqdm(cols2process):
#     ###step1
#     quantiles = df_train[col].value_counts()/len(df_train.index)
#     quantiles = quantiles.to_dict()
#     quantiles = {k: v for k, v in sorted(quantiles.items(), key=lambda item: item[0])}
#     ks = list(quantiles.keys())
#     vs = list(quantiles.values())
#     qs = np.cumsum(vs)
#     #print(ks)
#     #print(qs)
#     ###step2
#     qs = np.quantile(df_sub[col], qs)
#     #print(qs)
#     for i in range(len(qs)):
#         if i==0:
#             q = qs[0]
#             df_sub.loc[df_sub[col]<q, col] = ks[i]
#         elif i>0 and i<=len(qs)-1:
#             q0,q1 = qs[i-1], qs[i]
#             df_sub.loc[(df_sub[col]<q1)&(df_sub[col]>=q0), col] = ks[i]

In [21]:
# df_sub.head(10)

In [22]:
# df_sub.to_csv('submission.csv', index=False)